In [17]:
# 設定値欄

# 読みこむ列を定義する
VALID_ROW_TUPLE = ('H', 'J', 'L', 'N', 'T', 'V')

# 読みたいスプシのURL
URL = "https://docs.google.com/spreadsheets/d/17k6Gu-qIGzMFlGtAS5I7oU9P8n-b56r-XY6ONUvJam4/edit#gid=0"

# 取り出したい品詞
SELECT_CONDITIONS = ('動詞', '形容詞', '名詞')

# 指定文字以下のひらがなだけの単語を除外する
DISABEL_HIRAGANA_CONTINUE_LENGTH = 2

# 出力スプシ名
OUTPUT_SS_FILE_NAME = 'word_ranking'

# ファイル出力フォルダ名 勝手に作成されるので手動で作成する必要無し
DRIVE_ROOT_DIR = "./gdrive/My Drive/word_ranking/"

# エクセルの列名:インデックスマップ
COL_NUM_MAP = {
        "A":0,
        "B":1,
        "C":2,
        "D":3,
        "E":4,
        "F":5,
        "G":6,
        "H":7,
        "I":8,
        "J":9,
        "K":10,
        "L":11,
        "M":12,
        "N":13,
        "O":14,
        "P":15,
        "Q":16,
        "R":17,
        "S":18,
        "T":19,
        "U":20,
        "V":21,
        "W":22,
        "X":23,
        "Y":24,
        "Z":25,
        "AA":26,
        }


In [18]:
# 認証のためのコード 環境関係なく使える いじらないこと
from google.colab import auth
import gspread
from google.auth import default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

creds, _ = default()
gc = gspread.authorize(creds)
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
from google.colab import drive
drive.mount('./gdrive')


Drive already mounted at ./gdrive; to attempt to forcibly remount, call drive.mount("./gdrive", force_remount=True).


In [19]:
# 諸々インストール
! pip install pyfiglet
! pip install mecab-python3 unidic-lite
! apt-get -y install fonts-ipafont-gothic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-ipafont-gothic is already the newest version (00303-18ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [20]:
import datetime
from pyfiglet import Figlet
import shutil

# For WordCloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import codecs
from PIL import Image
import IPython
import sys
import os
from google.colab import files


# For MeCab
import re
from collections import Counter
import MeCab
import openpyxl
from matplotlib import pyplot as plt
from wordcloud import WordCloud

In [21]:
# 画像ファイルを全て削除する
try:
    shutil.rmtree(DRIVE_ROOT_DIR)
except FileNotFoundError:
    pass
os.mkdir(DRIVE_ROOT_DIR)

<h1>スプシ処理開始</h1>

In [22]:
# スプシを読む
ss = gc.open_by_url(URL)
st = ss.get_worksheet(0) 
list_of_row = st.get_all_values()

In [23]:
# 質問内容が解答の左列にあるので、それを取得して返す
# もらった数値の一個左列の一番上を見る
def get_question_content(index):
  tmp_value_list = st.get_all_values()
  return tmp_value_list[0][index-1]

In [24]:
# 列ごとにリストにして {質問内容: 値リスト}を生成
# ヘッダーは不要なので無視
col_value_list = {}
for reading_row_alpha in VALID_ROW_TUPLE:
  tmp_col_list = []
  for index, row in enumerate(list_of_row):
    if index == 0:
      continue
    tmp_col_list.append(row[COL_NUM_MAP[reading_row_alpha]])
  else:
    question = get_question_content(COL_NUM_MAP[reading_row_alpha])
    col_value_list[question] = tmp_col_list

In [25]:
# 読み込んだ列と値が正しそうなことを確認出来るエリア
for i, v in col_value_list.items():
  print(i, v)

弊社の「理念・哲学」の魅力を点数で評価してください。 ['顧客満足を大切にしていること', '行動指針が明確', '患者様の対応で困ったときに上司に伺ってもいつも患者様の特になるようにという答えが返ってくる。患者様を大事にしていると感じる。', '業界のイメージを変える、真面目に患者様のことを一番に考えるという点が、私自身の持っていた美容医療のイメージとも異なっており、会社の理念を大切にしているという点にも魅力を感じたため。', '他の美容クリニックとの差別化がすごく分かりやすく、大衆的でないところ、チープな感じではないところが理念、スローガンの通りで素晴らしいと思う。', '全スタッフが「患者様のために」「One Team」の精神で働いており、後輩スタッフにもそれを伝承しているのはとても魅力だと思います。', '顧客満足のために社内での対策、教育が充実している', '美容医療を通して、人々の人生をより良く豊かにしていけること。\nそして、そのことに常に真面目に誠実であり続けていよこと。この想いは今までもこれからとブレないこと。', '仕事だけでなく普段の生活にもクレドを活かせると思います。', '患者様第一にまじめに診療に取り組んでいること。', '「とことん真面目に、美容医療」というスローガンのもと、患者さまに寄り添うことができるから', '患者様のために、仲間のために組織全体で向上するための、チームワーク力を高める姿勢', 'スローガンである【とことん真面目に美容医療】が体現できている', 'クレドを指針として成長できる', '患者様第一という精神が浸透している', '美容クリニックには珍しい、患者様ファーストな考え方。', '「とことん真面目に美容医療」という点で患者様第一なところ', 'グループ・会社の指針がメンバーに浸透している', '患者様目線に立って高品質な美容医療を提供している。', '美容クリニックが多数ある中で売上でなく、患者様第一というところ', 'スローガン通り美容医療に対してとことん真面目に事業に取り組んでおり患者様が笑顔になるために誠実な姿勢でいるところ。', '今までの職場では理念などを聖心のように頻繁に振り返ることや深堀すること自体機会がなかったため、理念をスタッフに浸透させる働きそのものが率直に凄いなと感じます。\n多くの他院さん

<h1>Mecab処理エリア</h1>

In [26]:
# 分かち書き
tagger = MeCab.Tagger('')
tagger.parse('')
hiragana_re = re.compile('[\u3041-\u309F]+')
def wakati_text(input_text):
	node = tagger.parseToNode(input_text)
	terms = []

	while node:
		# 単語
		term = node.surface
		# 品詞
		pos = node.feature.split(',')[0]
		if pos in SELECT_CONDITIONS:
			is_hiragana_only = True
			for i in term:
				if not hiragana_re.match(i):
					is_hiragana_only = False
					break
			if is_hiragana_only and len(term) <= DISABEL_HIRAGANA_CONTINUE_LENGTH:
				node = node.next
				continue
			terms.append(term)
		node = node.next
	text_result = ' '.join(terms)
	return text_result, terms

In [27]:
# ワードクラウド生成
def generate_wordcloud(question_content, cloud_text):
  dt_now = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
  file_name = f'{question_content}.png'
  wordcloud = WordCloud(font_path = '/usr/share/fonts/truetype/fonts-japanese-mincho.ttf',                      
                       background_color="white",                      
                       width=1001,height=400).generate(cloud_text)
  plt.figure(figsize=(15,12))
  plt.imshow(wordcloud)
  plt.savefig(DRIVE_ROOT_DIR + file_name)

In [28]:
# スプシ書き込み
def write_col_value(col_aloha, input_text_list):
  st_name = col_aloha
  try:
    st2 = new_ss_obj.add_worksheet(title=st_name, rows=100, cols=10)
  except Exception as e:
    print(e)
  new_ss_obj.values_append(st_name, {"valueInputOption": "USER_ENTERED"}, {"values": input_text_list})

In [29]:
# スプシ生成
dt_now = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
dt_ymd = datetime.datetime.now().strftime('%Y%m%d')
is_created_new_ss = False
new_ss_obj = gc.create(f'{OUTPUT_SS_FILE_NAME}_{dt_now}')

In [30]:
# メイン関数に当たる
for question_content, col_value_list in col_value_list.items():
  text = ' '.join(col_value_list)
  joined_words, listed_words = wakati_text(text)
  listed_words.sort()
  counter_words = [list(i) for i in Counter(listed_words).most_common()]
  generate_wordcloud(question_content, joined_words)
  write_col_value(question_content, counter_words)

Output hidden; open in https://colab.research.google.com to view.

In [31]:
# シートを削除する。
try:
  new_ss_obj.del_worksheet(new_ss_obj.worksheet('シート1'))
except Exception as e:
  print(e)

In [32]:
f = Figlet(font="slant")
msg = f.renderText("DONE")
print(msg)

    ____  ____  _   ________
   / __ \/ __ \/ | / / ____/
  / / / / / / /  |/ / __/   
 / /_/ / /_/ / /|  / /___   
/_____/\____/_/ |_/_____/   
                            

